In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

from numpy import array
from numpy import max
import numpy as np
import pandas as pd
import math
import folium

In [2]:
from download import download

# We choose daily data
url = "https://opendata.arcgis.com/datasets/2ab16a5fb61f42c1a689fd9cc466383f_0.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=False)

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv'

## 1) Data treatment

In [3]:
occ_df = pd.read_csv(path_target)
occ_df = occ_df[occ_df['nom_poll'] == 'O3'] # only ozone

Que veut-on représenter? 
   +  Toutes les villes de notre data set en Occitanie
   +  La question est quand ?
   + Peut on faire varier la date interactivement
   + Voila pourquoi on veut une video, on va voire la pollution évoluer
   
J'ai reflechi cette nuit et on developpera ici:
   + Une map de l'occitanie qui affiche les données journalières
   + Elle sera interactive et on pourra choisir le jour qu'on veut
   + Il faudra certainement et au préalable nettoyer nos données

In [4]:
# On va traiter les données afin de rajouter une colonne de la date (jour)
occ_df['jour'] = pd.to_datetime(occ_df['date_debut']).dt.to_period('D')
# On rajoute une colonne pour avoir des valeurs standardisées
# Cela va servir à avoir une échelle des couleurs plus claire
occ_df['centrer_reduire'] = (occ_df[['valeur']] - np.mean(occ_df[['valeur']]))/ np.std(occ_df[['valeur']])

C:\Users\megan\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [5]:
occ_df.nom_station.unique()

array(['Mende - Vernede urbain', 'Saze - rural',
       'Toulouse-Berthelot Urbain', 'Montpellier Nord - Périurbain',
       'Toulouse-Mazades Urbain', 'Albi Urbain',
       'Montpellier - Prés d Arènes Urbain', 'Toulouse-Jacquier Urbain',
       'Lourdes-Lapaca Urbain', 'Castres-Travet Urbain',
       'Tarbes-Dupuy Urbain', 'Nîmes Sud - urbain',
       'La Calmette - Périurbain', 'Peyrusse Vieille Rural',
       'Montgiscard Urbain', 'Belesta en Lauragais Rural',
       'Montpellier Sud - Périurbain', 'Saint Estève - périurbain',
       'Agathois-Piscénois - Périurbain',
       'Biterrois-Narbonnais - Périurbain',
       'Saint Gaudens-Miramont Industriel', 'Rodez Urbain',
       'Caunes-Minervois - rural', 'Carcassonne - urbain',
       'Station Pedagogique', 'Perpignan Centre - urbain',
       'Roqueredonde - rurale', 'Biars-sur-Cere - urbain',
       'Vallabrègues - rural'], dtype=object)

In [6]:
occ_df.jour.unique() # 362 jours, on fera varier notre map selon ces jours.

<PeriodArray>
['2019-12-27', '2019-12-28', '2020-02-19', '2020-02-26', '2019-12-29',
 '2019-12-30', '2020-02-20', '2019-12-31', '2020-01-01', '2019-10-16',
 ...
 '2020-04-07', '2020-04-08', '2020-03-27', '2020-03-28', '2020-03-29',
 '2020-03-30', '2020-03-31', '2020-04-01', '2019-12-22', '2019-12-25']
Length: 363, dtype: period[D]

## 2) pollution map for one day 

On va prendre le tableau d'un jour en particulier pour commencer:

In [7]:
occ_day1 = occ_df[occ_df['jour'] == '2020-03-26']
occ_day1.nom_station.unique()

array(['Saze - rural', 'Toulouse-Mazades Urbain', 'Nîmes Sud - urbain',
       'Montgiscard Urbain', 'Belesta en Lauragais Rural',
       'Peyrusse Vieille Rural', 'Toulouse-Jacquier Urbain',
       'Saint Estève - périurbain', 'Castres-Travet Urbain',
       'Montpellier Sud - Périurbain',
       'Biterrois-Narbonnais - Périurbain',
       'Montpellier Nord - Périurbain', 'La Calmette - Périurbain',
       'Montpellier - Prés d Arènes Urbain', 'Lourdes-Lapaca Urbain',
       'Saint Gaudens-Miramont Industriel', 'Toulouse-Berthelot Urbain',
       'Station Pedagogique', 'Tarbes-Dupuy Urbain', 'Rodez Urbain',
       'Agathois-Piscénois - Périurbain', 'Caunes-Minervois - rural',
       'Perpignan Centre - urbain'], dtype=object)

In [8]:
# Ce qui nous interesse désormais dans ce tableau
# nom de la station et de la ville
# lattitude et longitude
# valeur Ozone

occ_day1 = occ_day1[['X','Y','nom_com', 'nom_station', 'valeur', 'jour', 'centrer_reduire']]

In [9]:
occ_day1 # nombre très raisonnable de données pour un jour

,X,Y,nom_com,nom_station,valeur,jour,centrer_reduire
47294,4.679690,43.9429,SAZE,Saze - rural,79.0,2020-03-26,0.768492
47297,1.438610,43.6236,TOULOUSE,Toulouse-Mazades Urbain,65.4,2020-03-26,0.166326
47298,4.374220,43.8344,NIMES,Nîmes Sud - urbain,74.9,2020-03-26,0.586957
47301,1.571110,43.4575,MONTGISCARD,Montgiscard Urbain,70.1,2020-03-26,0.374428
47302,1.821940,43.4414,BELESTA-EN-LAURAGAIS,Belesta en Lauragais Rural,74.4,2020-03-26,0.564818
47312,0.179722,43.6303,PEYRUSSE-VIEILLE,Peyrusse Vieille Rural,73.5,2020-03-26,0.524969
47324,1.418610,43.5756,TOULOUSE,Toulouse-Jacquier Urbain,68.8,2020-03-26,0.316868
51325,2.839870,42.7198,SAINT-ESTEVE,Saint Estève - périurbain,74.0,2020-03-26,0.547108
51327,2.230000,43.6106,CASTRES,Castres-Travet Urbain,56.2,2020-03-26,-0.241021
51353,3.911560,43.5706,LATTES,Montpellier Sud - Périurbain,72.7,2020-03-26,0.489548


In [10]:
import branca.colormap as cm

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=-3, vmax=1
)
# couleurs

In [11]:
map = folium.Map(location=[43.9333, 2.15], 
                 zoom_start=7.5, 
                 tiles='Stamen Terrain')

for i in range(0,len(occ_day1)):
   folium.Circle(
      location=[occ_day1.iloc[i]['Y'], occ_day1.iloc[i]['X']],
      popup=occ_day1.iloc[i]['nom_station'],
      radius=occ_day1.iloc[i]['valeur']*100,
      color='black',
      fill=True,
      fill_color=linear(occ_day1.iloc[i]['centrer_reduire']),
       fill_opacity=0.5,
       opacity=0.4,
   ).add_to(map)

map

# selectionner que certaines stations??


## 3) Interactive pollution map

In [12]:
from ipywidgets import interact  # widget manipulation
from IPython.display import HTML

In [13]:
# on doit dabord créer une fonction pour ensuite utiliser interact
# plus simple de choisir le mois puis le jour...
# faisons d'abord avec que le jour (ça fera beaucoup de jours)

def interactive_map(day = '2020-03-26'):
    
    occ_map = occ_df[occ_df['jour'] == day]
    
    map_int = folium.Map(location = [43.9333, 2.15], 
                         zoom_start = 7.5, 
                         tiles = 'Stamen Terrain')
    
    for i in range(0,len(occ_map)):
        folium.Circle(
            location = [occ_map.iloc[i]['Y'], occ_map.iloc[i]['X']],
            popup = occ_map.iloc[i]['nom_station'],
            radius = occ_map.iloc[i]['valeur']*100,
            color = 'black',
            fill = True,
            fill_color = linear(occ_map.iloc[i]['centrer_reduire']),
            fill_opacity = 0.5,
            opacity = 0.4,
        ).add_to(map_int)
    
    return(map_int)

In [14]:
interact(interactive_map, month=occ_df.jour.unique())

interactive(children=(Text(value='2020-03-26', description='day'), Output()), _dom_classes=('widget-interact',…

<function __main__.interactive_map(day='2020-03-26')>

In [15]:
import matplotlib.animation as animation
# Faire un truc interactif où on selectionne le mois et le jour
# Comment obtient on une image de ce truc ?
# Animons tout ça...